In [1]:
from torch.utils.data import DataLoader
from torch.nn import Embedding
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.vocab import Vocab
import torch

from datasets import load_dataset

/home/xd/miniconda3/envs/torch_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/xd/miniconda3/envs/torch_env/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/xd/miniconda3/envs/torch_env/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_

In [4]:
# Login using e.g. `huggingface-cli login` to access this dataset
train_dataset = load_dataset("ajaykarthick/imdb-movie-reviews", cache_dir="../../.datasets/", split="train")
test_dataset = load_dataset("ajaykarthick/imdb-movie-reviews", cache_dir="../../.datasets/", split="test")

In [5]:
print(train_dataset.info)
print(test_dataset.info)

DatasetInfo(description='', citation='', homepage='', license='', features={'review': Value('string'), 'label': Value('int64')}, post_processed=None, supervised_keys=None, builder_name='json', dataset_name='imdb-movie-reviews', config_name='default', version=0.0.0, splits={'train': SplitInfo(name='train', num_bytes=52981665, num_examples=40000, shard_lengths=None, dataset_name='imdb-movie-reviews'), 'test': SplitInfo(name='test', num_bytes=13101843, num_examples=10000, shard_lengths=None, dataset_name='imdb-movie-reviews')}, download_checksums={'hf://datasets/ajaykarthick/imdb-movie-reviews@2537b681735c256a7ab829666b118e03c3eb9e0a/train.jsonl': {'num_bytes': 53617757, 'checksum': None}, 'hf://datasets/ajaykarthick/imdb-movie-reviews@2537b681735c256a7ab829666b118e03c3eb9e0a/test.jsonl': {'num_bytes': 13260879, 'checksum': None}}, download_size=66878636, post_processing_size=None, dataset_size=66083508, size_in_bytes=132962144)
DatasetInfo(description='', citation='', homepage='', licens

In [6]:
tokenizer = get_tokenizer(tokenizer="basic_english", language="en")
def yield_tokens(data):
  for sample in data:
    yield tokenizer(sample["review"])

In [18]:
vocab: Vocab = build_vocab_from_iterator(iterator=yield_tokens(train_dataset), specials=["<unk>", "<pad>"])
vocab_size = len(vocab)
vocab.get_itos()
print("Vocab size:", vocab_size)
print("Index of 'movie':", vocab['movie'])
print("Word → Index:", vocab.get_stoi())  # stoi = string to index

Vocab size: 130309
Index of 'movie': 21
Word → Index: {'₤250': 130307, '₤100': 130306, '…until': 130305, '…but': 130304, '“you’ve': 130301, '“who’s': 130300, '“playboy”': 130295, '“next”': 130292, '“if': 130287, '“him”': 130286, '“family”': 130283, '“euro-cult”': 130281, '“consider': 130279, '“at': 130277, '‘showdown’': 130275, '‘lost': 130272, '‘humanization’': 130271, '‘enjoyable’': 130270, '‘depraved’': 130268, '‘a’': 130266, '‘anna': 130265, '–-': 130263, 'כרמון': 130262, 'þór': 130259, 'ýs': 130257, 'üvegtigris': 130254, 'ünfaithful': 130253, 'ünel': 130252, 'überwoman': 130251, 'über-babe': 130247, 'über-annoying': 130246, 'ø': 130244, 'ïts': 130242, 'ïn': 130241, 'évery': 130240, 'étoile': 130239, 'çetin': 130233, 'ça': 130232, 'åmål': 130230, 'äänekoski': 130229, 'ääliöt': 130228, 'ä': 130227, 'ángela': 130226, 'ánd': 130225, '¿acting': 130220, '½*': 130219, 'º': 130218, '·outdoor': 130216, '·ormond': 130215, '·fencing': 130213, '´dollman': 130209, '´cos': 130208, '®': 130206, 

In [19]:
embeddings = Embedding(num_embeddings=vocab_size, embedding_dim=200)

In [ ]:
class LSTM(torch.nn.Module):
  def __init__(self, vocab_size: int, embedding_dim:int, hidden1: int, num_layers: int, output_size: int):
    super().__init__()
    self.embedding = torch.nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
    self.rnn = torch.nn.LSTM(embedding_dim, hidden1, num_layers=num_layers, batch_first=True)
    self.fc1 = torch.nn.Linear(hidden1, output_size)

  
  def forward(self, data):
    embedded = self.embedding(data)
    out, (hidden, cell) = self.rnn(embedded)
    output = self.fc1(hidden[-1])

    return torch.sigmoid(output)

In [ ]:
model = LSTM(vocab_size=vocab_size, embedding_dim=200, hidden1=256, num_layers=1, output_size=2)